## 1) Free API selected from https://rapidapi.com/collection/list-of-free-apis

In [1]:
import requests

url = "https://imdb-top-100-movies1.p.rapidapi.com/"

headers = {
	"X-RapidAPI-Key": "08eb9fe3ecmsh2ffb2bb7096aef7p1026dcjsn41991636f012",
	"X-RapidAPI-Host": "imdb-top-100-movies1.p.rapidapi.com"
}

response = requests.get(url, headers=headers)

top_100 = response.json()

## 2) Exporting data via MySQL into database

### Importing libraries

In [2]:
from sqlalchemy import create_engine
import pymysql
import mysql.connector

In [3]:
cnx = mysql.connector.connect(user='root',
                              password='',
                              host='localhost',
                              database='imdb')

In [4]:
# Creating a cursor object
cursor = cnx.cursor()

### Establishing connections to mysql

In [5]:
engine = create_engine("mysql+pymysql://root:@localhost/imdb")

In [6]:
import pandas as pd

## 3) Fetching the data from API and moving it to Database

In [7]:
def get_movie_info(movie_input):
    title = ""
    try:
        title = movie_input['title']
    except:
        pass
    rating = ""
    try:
        rating = movie_input['rating']
    except:
        pass
    description = ""
    try:
        description = movie_input['description']
    except:
        pass
    rank = ""
    try:
        rank = movie_input['rank']
    except:
        pass
    return title, rating,description,rank

movies = list(map(get_movie_info,top_100))

In [8]:
df_movies = pd.DataFrame(movies,columns=["title","rating","description","rank"])

In [9]:
df_movies.to_sql(name="movies",con=engine,index=False, if_exists="replace")

100

In [10]:
def get_director_info(movie_input):  
    try:
        directors = movie_input['director'][0].split(", \n")
        names = []
        for director in directors:
            names.append(director)         
    except:
        pass  
    return names 

list_of_directors = list(map(get_director_info,top_100))

directors = []
for value in list_of_directors:
    if value in directors:
        pass
    else:
        try:
            for y in value:
                if y in directors:
                    pass
                else:
                    directors.append(y)
        except:
            directors.append(value)

In [11]:
df_directors = pd.DataFrame(directors,columns=["name"])

In [12]:
df_directors.to_sql(name="directors",con=engine,index=False, if_exists="replace")

72

In [13]:
# Adding ID columns as a primary key
query = ('ALTER TABLE movies ADD COLUMN `id` int(10) unsigned primary KEY AUTO_INCREMENT FIRST;')
cursor.execute(query)

query2 = ('ALTER TABLE directors ADD COLUMN `id` int(10) unsigned primary KEY AUTO_INCREMENT FIRST;')
cursor.execute(query2)

### Displaying the data from database as Pandas dataframe

In [14]:
df = pd.read_sql("SELECT * FROM Movies", engine)

In [15]:
df

id                                          title rating   
0     1                       The Shawshank Redemption    9.3  \
1     2                                  The Godfather    9.2   
2     3                                The Dark Knight    9.0   
3     4                               Schindler's List    9.0   
4     5  The Lord of the Rings: The Return of the King    9.0   
..  ...                                            ...    ...   
95   96                                        Vertigo    8.3   
96   97                                   Citizen Kane    8.3   
97   98                             North by Northwest    8.3   
98   99                                          Ikiru    8.3   
99  100                                              M    8.3   

                                          description  rank  
0   Over the course of several years, two convicts...     1  
1   The aging patriarch of an organized crime dyna...     2  
2   When the menace known as the Joker wreaks havo...     3  
3   In German-occupied Poland during World War II,...     4  
4   Gandalf and Aragorn lead the World of Men agai...     5  
..                                                ...   ...  
95  A former San Francisco police detective juggle...    96  
96  Following the death of publishing tycoon Charl...    97  
97  A New York City advertising executive goes on ...    98  
98  A bureaucrat tries to find meaning in his life...    99  
99  When the police in a German city are unable to...   100  

[100 rows x 5 columns]